# 📘 Day 3 – Co-visitation Matrix & Candidate Expansion

##  Introduction

In Day 3, we will build co-visitation matrices, a common technique in recommendation systems.
The idea:

If two items frequently appear together in the same user session,
they may be similar or often co-purchased/ co-clicked.

This day prepares candidates for retrieval (recall) in the next steps.

# 📦 1. Load Data (Chunk Loading for Safety)

In [34]:
import pandas as pd
import pyarrow.parquet as pq

path= "/kaggle/input/otto-recsys-dataset/train.parquet"
pf=pq.ParquetFile(path)

dfs=[]
for i,batch in enumerate(pf.iter_batches(batch_size=200000)):
    df = batch.to_pandas()
    dfs.append(df)
    print(f"Loaded chunk {i+1}")
    if i ==9:
        break

events =pd.concat(dfs, ignore_index=True)
events.head()

Loaded chunk 1
Loaded chunk 2
Loaded chunk 3
Loaded chunk 4
Loaded chunk 5
Loaded chunk 6
Loaded chunk 7
Loaded chunk 8
Loaded chunk 9
Loaded chunk 10


,aid,ts,type,session
0,1517085,1659304800025,0,0
1,1563459,1659304904511,0,0
2,1309446,1659367439426,0,0
3,16246,1659367719997,0,0
4,1781822,1659367871344,0,0


## 🔧 2. Preprocess Data

In [35]:
events =events.sort_values(["session","ts"])
events["ts"]=events["ts"].astype("int64")
events.head()

,aid,ts,type,session
0,1517085,1659304800025,0,0
1,1563459,1659304904511,0,0
2,1309446,1659367439426,0,0
3,16246,1659367719997,0,0
4,1781822,1659367871344,0,0


## 🧩 3. Build Co-visitation Matrix (Core Part)

•	 In a session, if commodity A and commodity B appear in the time window → consider that A and B have a co-present relationship.

•	 Count all sessions such pairs

•	 Finally, get a set of top-related products B for each A.

## 3.1 Define Helper Function

In [36]:
from collections import defaultdict
import numpy as np

def build_covis_matrix(df,event_type=None, time_window=30*60):
    """
    df:session-level dataframe
    event_type:'clicks','carts','orders',or None for all
    time_window:window in seconds for co-visitation
    """

    if event_type:
        df=df[df["type"] == event_type]

    session_groups =df.groupby("session")

    covis =defaultdict(lambda:defaultdict(float))

    for session_id, group in session_groups:
        item = group[["aid","ts"]].values

        for i in range(len(items)):
            aid_i,ts_i =items[i]

            for j in range(i+1,len(items)):
                aid_j,ts_j = items[j]

                if abs(ts_i - ts_j) <= time_window:

                    weight = 1.0
                    if event_type == "orders":
                        weight = 3.0
                    elif event_type == "carts":
                        weight = 2.0

                    covis[aid_i][aid_j] += weight
                    covis[aid_j][aid_i] += weight

    return covis

## 3.2 Build Co-visitation Matrics for All Types

In [37]:
covis_clicks = build_covis_matrix(events,event_type="clicks")
covis_carts = build_covis_matrix(events,event_type="carts")
covis_orders=build_covis_matrix(events,event_type="orders")

print("Done building co-visitation matrices.")

Done building co-visitation matrices.


## 🏆 4. Candidate Expansion (Top-40 Candidates)

In [38]:
def extract_topk(covis_dict,top_k=40):
    topk ={}
    for aid,neighbors in covis_dict.items():
        sorted_items = sorted(neighbors.items(),key=lambda x: -x[1])
        topk[aid] = [nid for nid, score in sorted_items[:top_40]]
    return topk

candidates_clicks = extract_topk(covis_clicks)
candidates_carts = extract_topk(covis_carts)
candidates_orders = extract_topk(covis_orders)

print("Candidate extraction completed.")

Candidate extraction completed.


## 💾 5. Merge Candidates (Final Expansion Dictionary)

In [39]:
final_candidates= {}

for dict_src in [candidates_clicks,candidates_carts,candidates_orders]:
    for aid,items in dict_src.items():
        if aid not in final_candidates:
            final_candidates[aid] = set()
        final_candidates[aid].update(items)

#Converted list
final_candidates ={k:list(v) for k ,v in final_candidates.items()}

print("Total items with candidates:",len(final_candidates))

Total items with candidates: 0


In [40]:
import json

with open("day3_candidates.json","w")as f:
    json.dump(final_candidates,f)

print("Saved day3_candidates.json")

Saved day3_candidates.json
